<a href="https://colab.research.google.com/github/marcocintra/Atmosphere/blob/master/CASE_STUDY_NAGOYA_MAPS_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ZIP FILES ADJUSTS

In [ ]:
!curl 'https://my.microsoftpersonalcontent.com/personal/a149071e7d7aaa61/_layouts/15/download.aspx?UniqueId=acd46945-a818-485d-a06b-eed3eaff8ad1&Translate=false&tempauth=v1e.eyJzaXRlaWQiOiI3NzM0Y2NkNy1mMWE2LTQ0ZmItOWZmYy03MjIyYzI4NGE0ODEiLCJhcHBpZCI6IjAwMDAwMDAwLTAwMDAtMDAwMC0wMDAwLTAwMDA0ODE3MTBhNCIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9teS5taWNyb3NvZnRwZXJzb25hbGNvbnRlbnQuY29tQDkxODgwNDBkLTZjNjctNGM1Yi1iMTEyLTM2YTMwNGI2NmRhZCIsImV4cCI6IjE3NDY1NzY0NDQifQ.5-mw7Q_m-rZOSw-VQU8Rh6C3zHe_LogaJQqkqCKgWWKiS6_ICamsAWSCmLIcVNghpC8pd8XgInB5I-7osAUfAiMoVOlivxbFceKHU0Qx1ADMV1KkEyH_t9AABNvSlGBMiS-hi15I3wTvGYG69_LsVm1O7yWDYhArp0FM7heb1-FdiOw_yrlezsUFMx5vMfoEkno6f2VRAHvNBqRXE-_0ssjwJ10vbRxsVC7U2frubwYrMEA1uQYrNCuVKpa9TXcT0w7_8mqI_ETGOU1aWwlngV_2BtgxhVUm1hQPX2NDDA_hBWh-YtuxVC7mT1yAjAIGiNzSESQsX5_B8B9OmfAQD43Xb3-_ngyGwdvKMLSMcIdAxe0OF9kjsxDNcnFrgmjyw5yLuTztq2lrSyLD_onBcaODso5zBKFUXhWrjILeasw.21DzhL9dc10xlNld6b5hbBsBsI84Hr5Os8upFdJi3zY&ApiVersion=2.0&AVOverride=1' \
  -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' \
  -H 'accept-language: pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7' \
  -H 'priority: u=0, i' \
  -H 'referer: https://onedrive.live.com/' \
  -H 'sec-ch-ua: "Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Linux"' \
  -H 'sec-fetch-dest: iframe' \
  -H 'sec-fetch-mode: navigate' \
  -H 'sec-fetch-site: cross-site' \
  -H 'sec-fetch-storage-access: none' \
  -H 'sec-fetch-user: ?1' \
  -H 'upgrade-insecure-requests: 1' \
  -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36' > 'NAGOYA-SEPT-20-2024-TO-END-SEPT-2024.zip'

In [ ]:
%pwd

In [ ]:
!ls

In [ ]:
!unzip -q 'NAGOYA-SEPT-20-2024-TO-END-SEPT-2024.zip'

# DATA FILES ADJUSTS

In [ ]:
!ls

In [ ]:
%cd NAGOYA-SEPT-20-2024-TO-END-SEPT-2024

In [ ]:
import os
import glob
from collections import defaultdict

base_path = './'

file_pattern = os.path.join(base_path, '**', '2024*.nc')
file_list = sorted(glob.glob(file_pattern, recursive=True))

file_count_by_day = defaultdict(int)

for file_name in file_list:

    day_dir = os.path.basename(os.path.dirname(file_name))
    file_count_by_day[day_dir] += 1

print("\nDias com menos de 144 arquivos:")

for day, count in sorted(file_count_by_day.items()):
    if count < 144:
        print(f"Dia {day}: {count} arquivos")

In [ ]:
import numpy as np
import xarray as xr
import os
import re
import glob

file_list = glob.glob("./2024*.nc")

pattern = re.compile(r'(\d{8})(\d{2})_atec')

def extract_datetime(file_name):
    match = pattern.search(file_name)
    if match:
        date = match.group(1)
        hour = match.group(2)
        return date, int(hour)
    return None, None

files_by_date = {}

for file_name in file_list:
    date, hour = extract_datetime(file_name)
    if date:
        if date not in files_by_date:
            files_by_date[date] = []
        files_by_date[date].append((file_name, hour))

sorted_dates = sorted(files_by_date.keys())
print(f"\nDatas a serem processadas em ordem cronológica: {sorted_dates}")

print("\nProcessando dados por dia (em ordem cronológica)...")
for date in sorted_dates:
    file_hour_pairs = files_by_date[date]

    file_hour_pairs.sort(key=lambda x: x[1])

    print(f"Processando o dia: {date}, {len(file_hour_pairs)} arquivos encontrados")

    final_data = np.zeros((288, 242, 221))

    for file_name, hour in file_hour_pairs:
        try:
            print(f"  Processando: {file_name}, Hora: {hour}")

            ds_disk = xr.open_dataset(file_name)
            tec = ds_disk['atec'][19:261, 140:361].values

            for timestep in range(12):

                global_idx = hour * 12 + timestep

                timestep_slice = tec[:, :, timestep]

                final_data[global_idx] = timestep_slice

            ds_disk.close()

        except Exception as e:
            print(f"  Erro ao processar o arquivo {file_name}: {e}")

    print(f"  Shape final do array: {final_data.shape}")

    output_file = f"atec{date}.npy"
    np.save(output_file, final_data)

    print(f"Arquivo {output_file} salvo com sucesso! Shape: {final_data.shape}")

print("\nProcessamento concluído!")

In [ ]:
!ls -lR ./*.npy | wc -l

In [ ]:
import numpy as np
import glob
import re
from datetime import datetime

npy_files = glob.glob("atec2024*.npy")

def extract_doy(filename):
    match = re.search(r'atec(\d{8})\.npy', filename)
    if match:
        date_str = match.group(1)
        try:
            date_obj = datetime.strptime(date_str, '%Y%m%d')
            return date_obj.timetuple().tm_yday
        except ValueError:
            return 0
    return 0

npy_files.sort(key=extract_doy)

print(f"Encontrados {len(npy_files)} arquivos NPY para processar.")

all_data = []

for i, file in enumerate(npy_files):
    doy = extract_doy(file)
    print(f"Processando {file} (DOY {doy}) - {i+1}/{len(npy_files)}")

    data = np.load(file)

    all_data.append(data)

    print(f"  Shape: {data.shape}")

combined_data = np.concatenate(all_data, axis=0)

print(f"\nDados combinados. Shape final: {combined_data.shape}")

output_file = "NAGOYA-SEPT-20-2024-AND-BEYOND.npy"
np.save(output_file, combined_data)

print(f"\nArquivo {output_file} salvo com sucesso!")
print(f"Tamanho do arquivo: {combined_data.nbytes / (1024**2):.2f} MB")

In [ ]:
!ls -lh *.npy

In [ ]:
from google.colab import files

In [ ]:
%pwd

In [ ]:
from google.colab import files

In [ ]:
!ls -lh NAGOYA*.npy

# DATES ADJUSTS

In [ ]:
import numpy as np

In [ ]:
nagoya_2024 = np.load('NAGOYA-SEPT-20-2024-AND-BEYOND.npy')

In [ ]:
import numpy as np
from datetime import datetime, timedelta

def doy_to_date(year, doy):
    """Convert year and day of year to datetime."""
    return datetime(year, 1, 1) + timedelta(days=doy - 1)

doys = {

    2024: {
        9: [264, 265, 266, 267, 268, 269, 270, 271, 272, 274],
        }
}

datetime_list = []

for year in doys.keys():
    for month in doys[year].keys():
        for doy in doys[year][month]:
            base_date = doy_to_date(year, doy)
            for hour in range(24):
                for minute in range(0, 60, 5):
                    dt = datetime(base_date.year, base_date.month, base_date.day, hour, minute, 0)
                    datetime_list.append(dt)

nagoya_datetime_2024 = np.array(datetime_list, dtype='datetime64[s]')

print(f"Total number of datetime points: {len(nagoya_datetime_2024)}")
print(f"First datetime: {nagoya_datetime_2024[0]}")
print(f"Last datetime: {nagoya_datetime_2024[-1]}")

np.save('nagoya_datetime_2024.npy', nagoya_datetime_2024)

print("\nSample of first 10 datetime entries:")
for dt in nagoya_datetime_2024[:10]:
    print(dt)

print("\nSample of last 10 datetime entries:")
for dt in nagoya_datetime_2024[-10:]:
    print(dt)

In [ ]:
nagoya_dates = np.load('nagoya_datetime_2024.npy')

In [ ]:
nagoya_dates.shape

In [ ]:
nagoya_dates

In [ ]:
nagoya_2024.shape

In [ ]:
print(len(nagoya_2024.tolist()))
print(len(nagoya_2024[0].tolist()))
print(len(nagoya_2024[0][0].tolist()))
# print(len(nagoya_2022_2024[0][0][0].tolist()))

In [ ]:
len(nagoya_2024[0:12])

In [ ]:
nagoya_2024_shaped = np.reshape(nagoya_2024, (-1, 242, 221))

In [ ]:
np.shape(nagoya_2024_shaped)

In [ ]:
np.shape(nagoya_2024_shaped.tolist())

## CREATE DATAFRAME

In [ ]:
nagoya_dates.shape

In [ ]:
import pandas as pd

data = {
    'DATETIME': nagoya_dates,
    'TECMAP': nagoya_2024_shaped.tolist()
}

df_mapas_nagoya_2024 = pd.DataFrame(data)

df_mapas_nagoya_2024

In [ ]:
np.array(df_mapas_nagoya_2024.iloc[0]['TECMAP'])

In [ ]:
np.array(df_mapas_nagoya_2024.iloc[0]['TECMAP']).shape

In [ ]:
df_mapas_nagoya_2024.to_pickle("./df_mapas_nagoya_sept_20_2024_to_end_sept_2024.pkl")

In [ ]:
mapas_nagoya_2024 = np.array(df_mapas_nagoya_2024.iloc[:]['TECMAP'])

In [ ]:
np.shape(df_mapas_nagoya_2024)

In [ ]:
np_mapas_nagoya_2024 = []
for i in range(len(mapas_nagoya_2024)):
    np_mapas_nagoya_2024.append(mapas_nagoya_2024[i])
np_mapas_nagoya_2024 = np.array(np_mapas_nagoya_2024)

In [ ]:
np.shape(np_mapas_nagoya_2024)

In [ ]:
type(np_mapas_nagoya_2024)

In [ ]:
np.save('mapas_nagoya_sept_20_2024_to_end_sept_2024.npy', np_mapas_nagoya_2024)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls -lh mapas*

In [ ]:
!ls -lh df_mapas*

In [ ]:
%cp /content/NAGOYA-SEPT-20-2024-TO-END-SEPT-2024/df_mapas_nagoya_sept_20_2024_to_end_sept_2024.pkl /content/drive/MyDrive/df_mapas_nagoya_sept_20_2024_to_end_sept_2024.pkl

In [ ]:
%cp /content/NAGOYA-SEPT-20-2024-TO-END-SEPT-2024/mapas_nagoya_sept_20_2024_to_end_sept_2024.npy /content/drive/MyDrive/mapas_nagoya_sept_20_2024_to_end_sept_2024.npy